<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/googlecolab_Spark_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=541b0f119648d31a67bae38caf51e4a30f32879ccbdfed74780c84934de75751
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [65]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [49]:
import requests

In [50]:
spark = SparkSession.builder.appName('practice').getOrCreate()

In [51]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
response = requests.get(url, stream=True)
with open("yellow_tripdata_2024-01.parquet", "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)



In [52]:
# Read the Parquet file from the local file system
df1 = spark.read.parquet("yellow_tripdata_2024-01.parquet")

In [53]:
df1.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [54]:
df1.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [55]:
df1.count()

2964624

In [58]:
df1.first()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 1, 1, 0, 57, 55), tpep_dropoff_datetime=datetime.datetime(2024, 1, 1, 1, 17, 43), passenger_count=1, trip_distance=1.72, RatecodeID=1, store_and_fwd_flag='N', PULocationID=186, DOLocationID=79, payment_type=2, fare_amount=17.7, extra=1.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=22.7, congestion_surcharge=2.5, Airport_fee=0.0)

In [59]:
df1.filter(df1.VendorID == 1).count()

729732

In [60]:
df1.groupBy('VendorID').count().show()

+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 729732|
|       2|2234632|
|       6|    260|
+--------+-------+



when mentioning groupy you dont have to mention select because they are always same set of columns.

In [61]:
df1.groupBy('VendorID').sum('tip_amount').orderBy('VendorID', ascending=False).show()

+--------+------------------+
|VendorID|   sum(tip_amount)|
+--------+------------------+
|       6|               0.0|
|       2| 7700953.170000889|
|       1|2188647.1400000826|
+--------+------------------+



In [62]:
arr = [[[2,3,4],'first row'],[[11,22,33],'second row'],[[111,222,333],'third row']]
error_df=spark.createDataFrame(arr,schema=["arr","row desc"])


In [63]:
error_df.show()

+---------------+----------+
|            arr|  row desc|
+---------------+----------+
|      [2, 3, 4]| first row|
|   [11, 22, 33]|second row|
|[111, 222, 333]| third row|
+---------------+----------+



In [66]:
error_df.select(f.explode('arr').alias('arr1_name'),"row desc").show()

+---------+----------+
|arr1_name|  row desc|
+---------+----------+
|        2| first row|
|        3| first row|
|        4| first row|
|       11|second row|
|       22|second row|
|       33|second row|
|      111| third row|
|      222| third row|
|      333| third row|
+---------+----------+



explode might be useful in dealing with JSON dataset where a column can have a array of value. and we want it to normalize.

In [67]:
error_df.cache()

DataFrame[arr: array<bigint>, row desc: string]